# TFIDF选取关键词

## 预设

In [2]:
import sys
import os
import openpyxl
import csv
import re
import pandas as pd
import jieba
import jieba.posseg as psg
import jieba.analyse as analyse
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import cn2an
import functools

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

### 全局常量

In [5]:
ExtraDictionaryPath = 'set/dict.txt'
StopWordPath = 'set/stopwords.txt'
KeywordTopNumber = 10
minLengthOfWord = 3
DataFilePath = 'data/5西安项目分类后二分词.pkl'
IDFFilePath = 'set/idf.txt'

## 数据读取

In [36]:
data = pd.read_pickle(DataFilePath)

In [7]:
data['二分词'].sample(3)

49078    建局 二手房 参考价 二手房 参考价格 建局 卖方 意愿 二手房 参考价格 问题 建局 工作...
17095                                                奶油 理光
29707    现场 教学 泡馍 全过程 新闻记者 泡馍 问题 技巧 视频 现场 教学 泡馍 国际 电影节 ...
Name: 二分词, dtype: object

## 处理函数工具

In [9]:
# 单篇文档提取关键词
def get_key_word_from_a_doc(content, topK=KeywordTopNumber):
    return ' '.join(analyse.extract_tags(content, topK))

In [12]:
def get_new_path(old_path, new_tag):
    path_list = old_path.split('.')
    path_list[0] += new_tag
    return '.'.join(path_list)

## 数据处理

### 获取总体关键字

In [48]:
corpus = data['二分词'].to_list()
# tfidf_vect = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, max_features=500, max_df = 0.05,
#                                 min_df = 15)
tfidf_vect = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, max_features=1000, max_df = 0.5,
                                min_df = 10)
tfidf_features = tfidf_vect.fit_transform(corpus)
keywords = tfidf_vect.get_feature_names()

#### 获取总体idf表

In [50]:
# idf_df_data = {'word':keywords}
idf_df_data = {'word':keywords,'idf':tfidf_vect.idf_}
idf_df = pd.DataFrame(idf_df_data)
display(idf_df)

,word
0,4s店
1,一带
2,丈夫
3,上学
4,上线
...,...
995,魅力
996,鲜城
997,鸡蛋
998,鼓楼


#### 保存idf表

In [51]:
idf_df.to_csv(IDFFilePath, header=None, index=None, sep=' ', mode='a')


### 获取个体关键字

In [44]:
analyse.set_idf_path(IDFFilePath);

In [45]:
get_key_word_from_a_doc(data['内容'][2])

'好吃 西安 真的 可以 这家 但是 因为 一个 现在 就是'

In [52]:
data['二词关键词'] = data['内容'].apply(get_key_word_from_a_doc)

## 数据展示&保存

In [16]:
df_to_show = data

In [35]:
display(df_to_show.sample(3)['二词关键词'])

47869    摄影 ## 守风人 凌安 DearWhite 不凡 亲爱 西安
55437                               小康家庭
376       秦岭 葛牌 石船 山沟 龙王庙 沟口 褚家 粉房 沟内 支沟
Name: 二词关键词, dtype: object

### 数据保存

In [18]:
df_to_save = data

In [19]:
new_path = get_new_path(DataFilePath, 'tfidf分词')


In [20]:
data.to_pickle(new_path)